# Análise sobre evolução da capacidade dos leitos durante a pandemia de Covid-19 no estado de Goiás

Esse projeto tem como fonte os dados obtidos através do site DataSUS, subsistema LT do sistema CNES (Cadastro Nacional de Estabelecimentos de Saúde), com intuito de responder as seguintes questões:

 - 1: Comparar distribuição geográfoca dos leitos por município antes e depois da pandemia
 - 2: Evolução do número de leitos durante os anos;
 - 3: Verificar declínio na capacidade hospitalar instalada;
 - 4: Comparar a quantidade de leitos SUS e não SUS no tempo e espaço.

 Dados obtidos através da biblioteca microdatasus.

 SALDANHA, Raphael de Freitas; BASTOS, Ronaldo Rocha; BARCELLOS, Christovam. Microdatasus: pacote para download e pré-processamento de microdados do Departamento de Informática do SUS (DATASUS). Cad. Saúde Pública, Rio de Janeiro , v. 35, n. 9, e00032419, 2019 . Available from http://ref.scielo.org/dhcq3y.

# 0.0 Imports

In [7]:
import os
import dbf
import glob

import pandas as pd
import numpy  as np


## 0.1. Loading Data

In [3]:
# Get list .dbc files
#dbc_files = glob.glob('*.dbc')

table = dbf.Table("../data/raw/CNES-LT-2020/LTGO2001.dbc")
table.open()

dbf_file = "LTGO2001."

#for dbc_file in dbc_files:
    # Open .dbc file
#    with open(dbc_file, 'rb') as f:
#        table = dbc

dbf.Table('../data/raw/CNES-LT-2020/LTGO2001.dbc', status=<DbfStatus.READ_ONLY: 1>)

In [11]:
path = '../data/raw/CNES-LT-2020/'

for file in os.listdir(path):
    if file.endswith('*.dbc'):
        table = dbf.Table(os.path.join(path, file))
        table.open()

        #df = pd.DataFrame(iter(table))
        df = pd.DataFrame(iter(table))

        table.close()

        df.to_csv(os.path.join(path, file.replace('.dbc', '.csv')), index = False)

In [ ]:
# Iterar sobre cada arquivo DBC e converter para DBF
for dbc_file in dbc_files:
    # Abrir o arquivo DBC
    with open(dbc_file, 'rb') as f:
        # Lê o conteúdo do arquivo DBC como um objeto dbf.Table
        table = dbf.Table(f)
        # Carregar os dados do objeto dbf.Table
        table.open()
        # Criar o nome do arquivo DBF com o mesmo nome do arquivo DBC, mas com a extensão alterada para .dbf
        dbf_file = dbc_file[:-3] + 'dbf'
        # Criar o arquivo DBF
        with open(dbf_file, 'wb') as f:
            # Escrever o conteúdo do objeto dbf.Table no arquivo DBF
            table.write(f)
        # Fechar o objeto dbf.Table
        table.close()

# 1.0. Data Description

# 2.0. Feature Engineering

# 3.0 Data Filtering

# 4.0. EDA

# 5.0. Dashboards